In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,MaxPooling3D, BatchNormalization)
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
x_train = np.load("x_train_3D.npy")
y_train = np.load("y_train_clef.npy")

x_train.shape, y_train.shape

((218, 128, 128, 32, 1), (218, 2))

In [3]:
INPUT_SHAPE = (128, 128, 32, 1)
nb_classes = 2

In [5]:
def ecir2020_v1():
    '''Simple 3D Convnet'''
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(INPUT_SHAPE), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Conv3D(32, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('softmax'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('softmax'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])
    model.summary()
    return model

def ecir2020_v2():
    '''3D Convnet wwith leaky relu activation'''
    model = None
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(INPUT_SHAPE), padding="same"))
    model.add(LeakyReLU())
    model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
    model.add(LeakyReLU())
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(LeakyReLU())
    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(LeakyReLU())
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(LeakyReLU())
    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(LeakyReLU())
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    return model
    
def ecir2020_v3():
    '''Convnet with batch normalization'''
    model = None
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(INPUT_SHAPE), padding="same"))
    model.add(Activation('relu'))
    model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(Activation('relu'))
    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(Activation('relu'))
    model.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    return model

In [7]:
model = None
model = ecir2020_v2()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 128, 128, 32, 32)  896       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128, 128, 32, 32)  0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 128, 128, 32, 32)  27680     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128, 128, 32, 32)  0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 43, 43, 11, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 43, 43, 11, 32)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 43, 43, 11, 64)    55360     
__________

In [8]:
x_train.shape, y_train.shape

((218, 128, 128, 32, 1), (218, 2))

In [ ]:
checkpointer = ModelCheckpoint(filepath='custom.h5', verbose=1, save_best_only=True)
h = model.fit(x_train, y_train, validation_split=0.1, batch_size=4, epochs=300, callbacks=[checkpointer])

In [ ]:
#model.save("custom_v2.h5")

In [ ]:
def plot_loss_accu(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'g')
    plt.plot(epochs, val_loss, 'y')
    plt.title('Training loss')
    plt.legend(['train', 'val'], loc='upper right')
    plt.grid()
    plt.show()
    loss = history.history['acc']
    val_loss = history.history['val_acc']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    plt.grid()
    plt.show()

In [ ]:
plot_loss_accu(h)

In [16]:
#from keras.models import load_model

#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')